In [20]:
# General imports
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score


#keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate

# Custom functions
%load_ext autoreload
%autoreload 2
import database_selection
import vectorization
import helpers
import icd9_cnn_model


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv('../data/disch_notes_all_icd9.csv',
                 names = ['HADM_ID', 'SUBJECT_ID', 'DATE', 'ICD9','TEXT'])

In [3]:
N_TOP = 20 
full_df, top_codes = database_selection.filter_top_codes(df, 'ICD9', N_TOP, filter_empty = True)
df = full_df.head(1000)

In [4]:
#preprocess icd9 codes
labels = vectorization.vectorize_icd_column(df, 'ICD9', top_codes)


In [5]:
#preprocess notes
MAX_VOCAB = None # to limit original number of words (None if no limit)
MAX_SEQ_LENGTH = 5000 # to limit length of word sequence (None if no limit)
df.TEXT = vectorization.clean_notes(df, 'TEXT')
data, dictionary, MAX_VOCAB = vectorization.vectorize_notes(df.TEXT, MAX_VOCAB, verbose = True)
data, MAX_SEQ_LENGTH = vectorization.pad_notes(data, MAX_SEQ_LENGTH)
print("Final Vocabulary: %s" % MAX_VOCAB)
print("Final Max Sequence Length: %s" % MAX_SEQ_LENGTH)

/home/zenla/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Vocabulary size: 22476
Average note length: 1748.878
Max note length: 5641
Final Vocabulary: 22476
Final Max Sequence Length: 5000


In [6]:
#split sets
X_train, X_val, X_test, y_train, y_val, y_test = helpers.train_val_test_split(
    data, labels, val_size=0.2, test_size=0.1, random_state=101)
print("Train: ", X_train.shape, y_train.shape)
print("Validation: ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

('Train: ', (699, 5000), (699, 20))
('Validation: ', (200, 5000), (200, 20))
('Test: ', (101, 5000), (101, 20))


In [7]:
# Delete temporary variables to free some memory
del df, data, labels

In [15]:
#creating embeddings
EMBEDDING_LOC = '../data/glove.6B.100d.txt' # location of embedding
EMBEDDING_DIM = 100 # given the glove that we chose
EMBEDDING_MATRIX, embedding_dict = vectorization.embedding_matrix(EMBEDDING_LOC,
                                                                  dictionary, EMBEDDING_DIM, verbose = True)


('Vocabulary in notes:', 22476)
('Vocabulary in original embedding:', 400000)
('Vocabulary intersection:', 14345)


## CNN for text classification

Based on the following papers and links:
* "Convolutional Neural Networks for Sentence Classification"   
* "A Sensitivity Analysis of (and Practitioners’ Guide to) Convolutional Neural Networks for Sentence Classification"
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://github.com/alexander-rakhlin/CNN-for-Sentence-Classification-in-Keras/blob/master/sentiment_cnn.py
* http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/
* https://github.com/dennybritz/cnn-text-classification-tf/blob/master/text_cnn.py

In [22]:
reload(icd9_cnn_model)
#### build model
model = icd9_cnn_model.build_icd9_cnn_model (input_seq_length=MAX_SEQ_LENGTH, max_vocab = MAX_VOCAB,
                             external_embeddings = True, embedding_trainable =True,
                             embedding_dim=EMBEDDING_DIM,embedding_matrix=EMBEDDING_MATRIX,
                             num_filters = 100, filter_sizes=[2,3,4,5],
                             training_dropout_keep_prob=0.9,
                             num_classes=N_TOP )

In [23]:
# Train the model
model.fit(X_train, y_train, batch_size=50, epochs=5,
validation_data=(X_val, y_val), verbose=2)

Train on 699 samples, validate on 200 samples
Epoch 1/5
48s - loss: 1.9359 - acc: 0.8041 - val_loss: 0.8085 - val_acc: 0.8207
Epoch 2/5
49s - loss: 1.6919 - acc: 0.8094 - val_loss: 0.7771 - val_acc: 0.8207
Epoch 3/5
49s - loss: 1.5507 - acc: 0.8093 - val_loss: 0.6493 - val_acc: 0.8238
Epoch 4/5
49s - loss: 1.3175 - acc: 0.8147 - val_loss: 0.6110 - val_acc: 0.8235
Epoch 5/5
49s - loss: 1.1382 - acc: 0.8146 - val_loss: 0.5787 - val_acc: 0.8235


In [24]:
pred_train = model.predict(X_train, batch_size=50)
pred_dev = model.predict(X_val, batch_size=50)

## Performance Evaluation

In [25]:
def get_f1_score(y_true,y_hat,threshold, average):
    hot_y = np.where(np.array(y_hat) > threshold, 1, 0)
    return f1_score(np.array(y_true), hot_y, average=average)

print 'F1 scores'
print 'threshold | training | dev  '
f1_score_average = 'micro'
for threshold in [ 0.02, 0.03,0.04,0.05,0.055,0.058,0.06, 0.08, 0.1,0.2,0.3, 0.5]:
    train_f1 = get_f1_score(y_train, pred_train,threshold,f1_score_average)
    dev_f1 = get_f1_score(y_val, pred_dev,threshold,f1_score_average)
    print '%1.3f:      %1.3f      %1.3f' % (threshold,train_f1, dev_f1)

F1 scores
threshold | training | dev  
0.020:      0.346      0.351
0.030:      0.372      0.368
0.040:      0.373      0.359
0.050:      0.363      0.364
0.055:      0.362      0.367
0.058:      0.363      0.369
0.060:      0.364      0.372
0.080:      0.371      0.382
0.100:      0.346      0.354
0.200:      0.199      0.204
0.300:      0.176      0.162
0.500:      0.002      0.000


### TO DO: model for Thresholding
Papers about Thresholding:    

* "Convolutional Neural Network using a Threshold Predictor for Multi-label Speech Act Classification"   
* "A Study on Threshold Selection for Multi-label Classification"   

These papers mention they use a model for thresholding, but it is not the main topic:   
* "A Review on Multi-Label Learning Algorithms"   
* "Large-scale Multi-label Text Classification—Revisiting Neural Networks"*   
* "A multi-label convolutional neural network for automatic image annotation"

### Results with external embeddings = True , no additional training,  top 20
```
F1 scores
threshold | training | dev  
0.020:      0.337      0.329
0.030:      0.360      0.353
0.040:      0.365      0.374
0.050:      0.372      0.375
0.055:      0.370      0.377
0.058:      0.369      0.375
0.060:      0.368      0.375
0.080:      0.348      0.361
0.100:      0.309      0.319
0.200:      0.198      0.208
0.300:      0.157      0.138
0.500:      0.000      0.000
```

### Results with external embeddings = False, top 20
```
F1 scores
threshold | training | dev  
0.020:      0.288      0.300
0.030:      0.327      0.322
0.040:      0.371      0.363
0.050:      0.380      0.391
0.055:      0.412      0.383
0.058:      0.403      0.394
0.060:      0.394      0.389
0.080:      0.385      0.390
0.100:      0.229      0.225
0.200:      0.000      0.000
0.300:      0.000      0.000
0.500:      0.000      0.000
```

### Results with external embedding and training them , top 20
```
F1 scores
threshold | training | dev  
0.020:      0.334      0.333
0.030:      0.362      0.360
0.040:      0.366      0.374
0.050:      0.373      0.380
0.055:      0.374      0.382
0.058:      0.376      0.376
0.060:      0.376      0.378
0.080:      0.387      0.371
0.100:      0.366      0.350
0.200:      0.179      0.171
0.300:      0.020      0.020
0.500:      0.000      0.000

```

### Results with external Embeddings = False, top 10, 
We can compare this setup with the LSTM published in the paper "Applying Deep Learning to ICD-9 Multi-label Classification from Medical Records", they got a F1-score of about 0.4168, we are getting 0.447

``` 
F1 scores
threshold | training | dev  
0.020:      0.399      0.407
0.030:      0.399      0.407
0.040:      0.399      0.407
0.050:      0.408      0.413
0.055:      0.433      0.420
0.058:      0.437      0.430
0.060:      0.432      0.427
0.080:      0.501      0.463
0.100:      0.446      0.463
0.200:      0.206      0.066
0.300:      0.000      0.000
0.500:      0.000      0.000
```



## Notes:


(1) There is a LSTM model by this paper: "Applying Deep Learning to ICD-9 Multi-label Classification from Medical Records" which did achieve a 42% F1-score. (https://cs224d.stanford.edu/reports/priyanka.pdf), but it only uses the top 10 icd9 codes. We are getting 46% (just running with 1000 notes so far)


(2) The "A Comparison of Rule-Based and Deep Learning Models for Patient Phenotyping"  study did get a 70% F1-score, but they don't use the icd9-labels but phenotypes labels they annotated themselved (via a group of medical professionals). (https://arxiv.org/abs/1703.08705). There were ONLY 10 phenotypes.

The discharge summaries are labeled with ICD9-codes that are leaves in the ICD9-hierarchy (which has hundreds of ICD9-codes), then maybe these leave nodes are too specific and difficult to predict, one experiment would be to replaced all the ICD9-codes with their parent in the second or third level in the hierarchy and see if predictions work better that way.   

(3) our baseline with top 20 codes had a f1-score of 35% (assigning top 4 icd9 codes to all notes, using a CNN with no external embeddings is getting about 40% f1-score.. a little better than the baseline

(4) Papers published and best practices report  that external embeddings improve considerable the model's performance.. maybe it is not the case here because of the medical terms..   

(5) Fixed Thresholding doesn't work well for multilabel classifications, we can implement a model to choose the appropriate threshold for each record (see notes above)